In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
import time

from selenium.webdriver.common.by import By
from xlwt import Workbook

import requests
from bs4 import BeautifulSoup
from xlwt import Workbook
from lxml import etree
import pandas as pd

In [189]:
wb = Workbook()  # Prepare to save to spreadsheet. NEED TO FIX
state_ballot_measures = wb.add_sheet('State Ballot Measures')
state_ballot_measures.write(0, 0, "Year");
state_ballot_measures.write(0, 1, "State");
state_ballot_measures.write(0, 2, "Type of measure");
state_ballot_measures.write(0, 3, "Name");
state_ballot_measures.write(0, 4, "Link");
state_ballot_measures.write(0, 5, "Support contributions");
state_ballot_measures.write(0, 6, "Oppose contributions");
state_ballot_measures.write(0, 7, "Yes votes");
state_ballot_measures.write(0, 8, "No votes");
state_ballot_measures.write(0, 9, "Title?");
state_ballot_measures.write(0, 10, "Ballot title");
state_ballot_measures.write(0, 11, "Ballot question");
state_ballot_measures.write(0, 12, "Short ballot title");
state_ballot_measures.write(0, 13, "Long ballot title");
state_ballot_measures.write(0, 14, "Summary?");
state_ballot_measures.write(0, 15, "Ballot summary");
state_ballot_measures.write(0, 16, "Summary and analysis");
state_ballot_measures.write(0, 17, "Republican presidential vote");
state_ballot_measures.write(0, 18, "Democratic presidential vote");



row = 0
for year in range(2016, 2018):  # do it year by year!
    url = 'https://ballotpedia.org/Ballot_measure_campaign_finance,_' + str(year)

    current_page = requests.get(url)
    current_soup = BeautifulSoup(current_page.content, 'html.parser')
    
    leg_ref = current_soup.find(id="Legislative_and_automatic_referrals")
    table_start = leg_ref.find_parent().find_next_sibling().find_next_sibling()
    body = table_start.findChild("tbody", recursive=False)
    row_count = len(body.findChildren("td"))
    for i in range(row_count // 4):
        row += 1
        
        print(row)
        state_ballot_measures.write(row, 0, year);
        state_ballot_measures.write(row, 2, "Legislative and automatic referrals");
        
        time.sleep(0.25)
        
        # get name and finance
        state_ballot_measures.write(row, 3, body.findChildren("td")[4 * i].text);
        state_ballot_measures.write(row, 5, body.findChildren("td")[4 * i + 1].text);
        state_ballot_measures.write(row, 6, body.findChildren("td")[4 * i + 2].text);
        
        # get page
        initiative_url = "https://ballotpedia.org" + body.findChildren("td")[4 * i].findChild()['href']
        state_ballot_measures.write(row, 4, initiative_url);
        print(initiative_url)
        if (initiative_url == 'https://ballotpedia.org/Alabama_Amendment_1,_Appointed_Education_Board_Amendment_(March_2020)'):
            continue
        initiative_page = requests.get(initiative_url)
        initiative_soup = BeautifulSoup(initiative_page.content, 'html.parser')
        
        # get results
        results = initiative_soup.find(id="Election_results")
        results_start = results.find_parent().find_next_sibling()
        while (str(results_start)[0:6] != "<table"):
            results_start = results_start.find_next_sibling()
            
        state_ballot_measures.write(row, 7, results_start.findChildren("td")[4].text);
        state_ballot_measures.write(row, 8, results_start.findChildren("td")[7].text);
        
        # get ballot title
        has_title = "No"
        ballot_title = initiative_soup.find(id="Ballot_title")  # doesn't always exist! might be "ballot question"
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 10, title_text.text[5:-5]);
            has_title = "Yes"
            
        ballot_title = initiative_soup.find(id="Ballot_question")
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            
            state_ballot_measures.write(row, 11, title_text.text[5:-5]);
            has_title = "Yes"
        
        short_ballot_title = initiative_soup.find(id="Short_ballot_title")
        long_ballot_title = initiative_soup.find(id="Long_ballot_title")
        if (short_ballot_title is not None and long_ballot_title is not None):
            short_title_text = short_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 12, title_text.text[5:-5]);
            long_title_text = long_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 13, title_text.text[5:-5]);
            
            has_title = "Yes"
        
        state_ballot_measures.write(row, 9, has_title);
        
        # get ballot summary
        has_summary = "No"
        ballot_summary = initiative_soup.find(id="Ballot_summary")
        if (ballot_summary is not None):
            summary_text = ballot_summary.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 15, summary_text.text[5:-5]);
            has_summary = "Yes"
            
        summary_and_analysis = initiative_soup.find(id="Summary_and_analysis")
        if (summary_and_analysis is not None):
            summary_and_analysis_text = summary_and_analysis.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 16, summary_and_analysis_text.text[5:-5]);
            has_summary = "Yes"
            
        state_ballot_measures.write(row, 14, has_summary);
        
    init = current_soup.find(id="Initiatives")
    table_start = init.find_parent().find_next_sibling().find_next_sibling()
    body = table_start.findChild("tbody", recursive=False)
    row_count = len(body.findChildren("td"))
    for i in range(row_count // 4):
        row += 1
        
        print(row)
        
        
        state_ballot_measures.write(row, 0, year);
        state_ballot_measures.write(row, 2, "Initiatives and veto referendums");
        
        time.sleep(0.25)
        
        # get name and finance
        state_ballot_measures.write(row, 3, body.findChildren("td")[4 * i].text);
        state_ballot_measures.write(row, 5, body.findChildren("td")[4 * i + 1].text);
        state_ballot_measures.write(row, 6, body.findChildren("td")[4 * i + 2].text);
        
        # get page
        initiative_url = body.findChildren("td")[4 * i].findChild()['href']
        print(initiative_url)
        initiative_page = requests.get(initiative_url)
        initiative_soup = BeautifulSoup(initiative_page.content, 'html.parser')
        
        # get results
        results = initiative_soup.find(id="Election_results")
        results_start = results.find_parent().find_next_sibling()
        while (str(results_start)[0:6] != "<table"):
            results_start = results_start.find_next_sibling()
        
        state_ballot_measures.write(row, 7, results_start.findChildren("td")[4].text);
        state_ballot_measures.write(row, 8, results_start.findChildren("td")[7].text);
        
        # get ballot title
        has_title = "No"
        ballot_title = initiative_soup.find(id="Ballot_title")  # doesn't always exist! might be "ballot question"
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 10, title_text.text[5:-5]);
            has_title = "Yes"
            
        ballot_title = initiative_soup.find(id="Ballot_question")
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 11, title_text.text[5:-5]);
            has_title = "Yes"
        
        short_ballot_title = initiative_soup.find(id="Short_ballot_title")
        long_ballot_title = initiative_soup.find(id="Long_ballot_title")
        if (short_ballot_title is not None and long_ballot_title is not None):
            short_title_text = short_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 12, title_text.text[5:-5]);
            long_title_text = long_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 13, title_text.text[5:-5]);
            
            has_title = "Yes"
        
        state_ballot_measures.write(row, 9, has_title);
        
        # get ballot summary
        has_summary = "No"
        ballot_summary = initiative_soup.find(id="Ballot_summary")
        if (ballot_summary is not None):
            summary_text = ballot_summary.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 15, summary_text.text[5:-5]);
            has_summary = "Yes"
            
        summary_and_analysis = initiative_soup.find(id="Summary_and_analysis")
        if (summary_and_analysis is not None):
            summary_and_analysis_text = summary_and_analysis.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 16, summary_and_analysis_text.text[5:-5]);
            has_summary = "Yes"
            
        state_ballot_measures.write(row, 14, has_summary);    
        
    veto = current_soup.find(id="Veto_referendums")
    table_start = veto.find_parent().find_next_sibling().find_next_sibling()
    body = table_start.findChild("tbody", recursive=False)
    row_count = len(body.findChildren("td"))
    for i in range(row_count // 4):
        row += 1
        
        print(row)
        
        
        state_ballot_measures.write(row, 0, year);
        state_ballot_measures.write(row, 2, "Initiatives and veto referendums");
        
        time.sleep(0.25)
        
        # get name and finance
        state_ballot_measures.write(row, 3, body.findChildren("td")[4 * i].text);
        state_ballot_measures.write(row, 5, body.findChildren("td")[4 * i + 1].text);
        state_ballot_measures.write(row, 6, body.findChildren("td")[4 * i + 2].text);
        
        # get page
        initiative_url = body.findChildren("td")[4 * i].findChild()['href']
        print(initiative_url)
        initiative_page = requests.get(initiative_url)
        initiative_soup = BeautifulSoup(initiative_page.content, 'html.parser')
        
        # get results
        results = initiative_soup.find(id="Election_results")
        results_start = results.find_parent().find_next_sibling()
        while (str(results_start)[0:6] != "<table"):
            results_start = results_start.find_next_sibling()
        
        state_ballot_measures.write(row, 7, results_start.findChildren("td")[4].text);
        state_ballot_measures.write(row, 8, results_start.findChildren("td")[7].text);
        
        # get ballot title
        has_title = "No"
        ballot_title = initiative_soup.find(id="Ballot_title")  # doesn't always exist! might be "ballot question"
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 10, title_text.text[5:-5]);
            has_title = "Yes"
            
        ballot_title = initiative_soup.find(id="Ballot_question")
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 11, title_text.text[5:-5]);
            has_title = "Yes"
        
        short_ballot_title = initiative_soup.find(id="Short_ballot_title")
        long_ballot_title = initiative_soup.find(id="Long_ballot_title")
        if (short_ballot_title is not None and long_ballot_title is not None):
            short_title_text = short_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 12, title_text.text[5:-5]);
            long_title_text = long_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 13, title_text.text[5:-5]);
            
            has_title = "Yes"
        
        state_ballot_measures.write(row, 9, has_title);
        
        # get ballot summary
        has_summary = "No"
        ballot_summary = initiative_soup.find(id="Ballot_summary")
        if (ballot_summary is not None):
            summary_text = ballot_summary.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 15, summary_text.text[5:-5]);
            has_summary = "Yes"
            
        summary_and_analysis = initiative_soup.find(id="Summary_and_analysis")
        if (summary_and_analysis is not None):
            summary_and_analysis_text = summary_and_analysis.find_parent().find_next_sibling().find_next_sibling()
            state_ballot_measures.write(row, 16, summary_and_analysis_text.text[5:-5]);
            
            has_summary = "Yes"
            
        state_ballot_measures.write(row, 14, has_summary);    
    
    excel_title = "State initiatives " + str(year) + ".xls"
    wb.save(excel_title)

1
https://ballotpedia.org/Washington_Modifying_Tax_Exemption_Criteria_for_Alternative_Fuel_Vehicles,_Advisory_Vote_15_(2016)
2


TypeError: 'NoneType' object is not subscriptable

In [71]:
url = "https://ballotpedia.org/List_of_ballot_measures_by_topic"
current_page = requests.get(url)
current_soup = BeautifulSoup(current_page.content, 'html.parser')

table = current_soup.find('tr')
topics = table.find_all("a", href=True)

[<a href="/Hunting_and_fishing_on_the_ballot" title="Hunting and fishing on the ballot">Hunting and fishing on the ballot</a>, <a href="/Treatment_of_animals_on_the_ballot" title="Treatment of animals on the ballot">Treatment of animals on the ballot</a>, <a href="/Adult_entertainment_on_the_ballot" title="Adult entertainment on the ballot">Adult entertainment on the ballot</a>, <a href="/Alcohol_on_the_ballot" title="Alcohol on the ballot">Alcohol on the ballot</a>, <a href="/Banking_on_the_ballot" title="Banking on the ballot">Banking on the ballot</a>, <a href="/Business_regulation_on_the_ballot" title="Business regulation on the ballot">Business regulation on the ballot</a>, <a href="/Definition_of_a_corporation_on_the_ballot" title="Definition of a corporation on the ballot">Definition of a corporation on the ballot</a>, <a href="/Economic_development_on_the_ballot" title="Economic development on the ballot">Economic development on the ballot</a>, <a href="/Food_and_agriculture_on

In [99]:
measure_categories = []

for topic in topics:
    topic_link = 'https://ballotpedia.org' + topic['href']
    topic_page = requests.get(topic_link);
    topic_soup = BeautifulSoup(topic_page.content, 'html.parser')
    
    measures_div = topic_soup.find("td", valign="top")
    measures = measures_div.find_all("a", href=True)
    
    category = {"topic": topic.text, "measures": []}
    for measure in measures:
        category["measures"].append(measure['href'])
    
    measure_categories.append(category)

In [101]:
for i in range(len(measure_categories)):
    category = measure_categories[i]
    good_measures = []
    for j in range(len(category["measures"])):
        next_url = "https://ballotpedia.org" + category["measures"][j]
        year = next_url[-5:-1]
        
        try:
            year = int(year)
        except:
            continue

        if (year > 2021 or year < 2008):
            continue

        if (measure_soup.find(id="Results") is not None) and (measure_soup.find(id="Election_results") is not None):
            continue
            
        good_measures.append(next_url)
    measure_categories[i]["measures"] = good_measures

In [187]:
wb = Workbook()  # Prepare to save to spreadsheet. NEED TO FIX
local_ballot_measures = wb.add_sheet('State Ballot Measures')
local_ballot_measures.write(0, 0, "Year");
local_ballot_measures.write(0, 1, "State");
local_ballot_measures.write(0, 2, "Category");
local_ballot_measures.write(0, 3, "Name");
local_ballot_measures.write(0, 4, "Link");
local_ballot_measures.write(0, 5, "Support contributions");
local_ballot_measures.write(0, 6, "Oppose contributions");
local_ballot_measures.write(0, 7, "Yes votes");
local_ballot_measures.write(0, 8, "No votes");
local_ballot_measures.write(0, 9, "Title?");
local_ballot_measures.write(0, 10, "Ballot title");
local_ballot_measures.write(0, 11, "Ballot question");
local_ballot_measures.write(0, 12, "Short ballot title");
local_ballot_measures.write(0, 13, "Long ballot title");
local_ballot_measures.write(0, 14, "Summary?");
local_ballot_measures.write(0, 15, "Ballot summary");
local_ballot_measures.write(0, 16, "Summary and analysis");
local_ballot_measures.write(0, 17, "Republican presidential vote");
local_ballot_measures.write(0, 18, "Democratic presidential vote");
local_ballot_measures.write(0, 19, "Support");
local_ballot_measures.write(0, 20, "R- Support");
local_ballot_measures.write(0, 21, "D- Support");
local_ballot_measures.write(0, 22, "Opposition");
local_ballot_measures.write(0, 23, "R- Opposition");
local_ballot_measures.write(0, 24, "D- Opposition");





row = 0
for category in measure_categories:
    for ballot_measure in category["measures"]:
        print(ballot_measure)
        row += 1
        
        initiative_page = requests.get(ballot_measure)
        initiative_soup = BeautifulSoup(initiative_page.content, 'html.parser')

        local_ballot_measures.write(row, 2, category["topic"])
        
        # get results
        local_ballot_measures.write(row, 4, ballot_measure)
        if (ballot_measure == "https://ballotpedia.org/North_Dakota_Captive_Game_Animal_Killing_Prohibited_Initiative,_Initiated_Statutory_Measure_2_(2010)" or ballot_measure == 'https://ballotpedia.org/Arizona_Property_Tax_Break_For_Business_Equipment_Amendment,_Proposition_116_(2012)' or ballot_measure == 'https://ballotpedia.org/Arkansas_Casino_Amendment,_Issue_3_and_4_(2012)' or ballot_measure == 'https://ballotpedia.org/Georgia_Sales_Tax_Increase_to_Fund_Transportation_Projects,_Referendum_1_(July_2012)' or ballot_measure == 'https://ballotpedia.org/Georgia_Democratic_Primary_Advisory_Questions_(May_2014)'):
            continue
        
        results = initiative_soup.find(id="Election_results")
        if (results is None):
            continue
        results_start = results.find_parent().find_next_sibling()
        while (str(results_start)[0:6] != "<table"):
            print("Here!2")
            print(str(results_start)[0:6])
            results_start = results_start.find_next_sibling()
            
        if (results_start.findChildren("td")[6].text == "No" or results_start.findChildren("td")[3].text == "Yes" or results_start.findChildren("td")[6].text[2:] == "No" or results_start.findChildren("td")[3].text[2:] == "Yes"):
            print("WOO")
            local_ballot_measures.write(row, 7, results_start.findChildren("td")[4].text);
            local_ballot_measures.write(row, 8, results_start.findChildren("td")[7].text);
        else:
            print("WAH")
            local_ballot_measures.write(row, 8, results_start.findChildren("td")[4].text);
            local_ballot_measures.write(row, 7, results_start.findChildren("td")[7].text);
            
        # get ballot title
        has_title = "No"
        ballot_title = initiative_soup.find(id="Ballot_title")  # doesn't always exist! might be "ballot question"
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 10, title_text.text);
            has_title = "Yes"

        ballot_title = initiative_soup.find(id="Ballot_question")
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 11, title_text.text);
            has_title = "Yes"

        short_ballot_title = initiative_soup.find(id="Short_ballot_title")
        long_ballot_title = initiative_soup.find(id="Long_ballot_title")
        if (short_ballot_title is not None and long_ballot_title is not None):
            short_title_text = short_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 12, title_text.text);
            long_title_text = long_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 13, title_text.text);

            has_title = "Yes"

        local_ballot_measures.write(row, 9, has_title);
        
        # get ballot summary
        has_summary = "No"
        ballot_summary = initiative_soup.find(id="Ballot_summary")
        if (ballot_summary is not None):
            summary_text = ballot_summary.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 15, summary_text.text);
            has_summary = "Yes"
            
        summary_and_analysis = initiative_soup.find(id="Summary_and_analysis")
        if (summary_and_analysis is not None):
            summary_and_analysis_text = summary_and_analysis.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 16, summary_and_analysis_text.text);
            has_summary = "Yes"

        local_ballot_measures.write(row, 14, has_summary);
        
        # get support and opposition
        # find support/opposition section first. add text from adjacent sections until reaching next h2
        support = initiative_soup.find(id="Support")
        support_text = ""
        if support is not None:
            support = support.find_parent().find_next_sibling()
            while support is not None and str(support)[0:3] != "<h2":
                support_text += support.text;
                support = support.find_next_sibling()
                
        local_ballot_measures.write(row, 19, len(support_text));
        local_ballot_measures.write(row, 20, support_text.count('R-') + support_text.count('(R)'));
        local_ballot_measures.write(row, 21, support_text.count('D-') + support_text.count('(D)'));
        
        longest_opposition_link = "";
        opposition = initiative_soup.find(id="Opposition")
        opposition_text = ""
        if opposition is not None:
            opposition = opposition.find_parent().find_next_sibling()
            while opposition is not None and str(opposition)[0:3] != "<h2":
                if len(opposition_text + opposition.text) < 30000:
                    opposition_text += opposition.text;
                    opposition = opposition.find_next_sibling()
                else:
                    longest_opposition_link = ballot_measure
                    break
                    
        local_ballot_measures.write(row, 22, len(opposition_text));
        local_ballot_measures.write(row, 23, opposition_text.count('R-') + opposition_text.count('(R)'));
        local_ballot_measures.write(row, 24, opposition_text.count('D-'));

wb.save("State initiatives with support.xls")

https://ballotpedia.org/Alabama_Right_to_Hunt_and_Fish,_Amendment_5_(2014)
Here!2
<p>Bel
WOO
4146
87
32
380
0
3
https://ballotpedia.org/Alaska_Wolf_and_Bear_Protection,_Measure_2_(August_2008)
WAH
1096
0
0
1521
1
0
https://ballotpedia.org/Arizona_Hunting_Amendment,_Proposition_109_(2010)
Here!2
<dl><d
Here!2
<p>Off
WAH
5833
0
0
4788
0
0
https://ballotpedia.org/Arkansas_Hunting_Rights,_Proposed_Amendment_1_(2010)
Here!2
<dl><d
WOO
1090
0
0
1301
0
0
https://ballotpedia.org/Idaho_Hunting_and_Fishing_Amendment,_HJR_2_(2012)
Here!2
<dl><d
Here!2
<p>The
WOO
716
0
0
1549
0
0
https://ballotpedia.org/Indiana_Right_to_Hunt_and_Fish,_Public_Question_1_(2016)
WOO
1663
10
0
1860
1
2
https://ballotpedia.org/Kansas_Right_to_Hunt_and_Fish,_Constitutional_Amendment_1_(2016)
WOO
1946
4
2
890
0
2
https://ballotpedia.org/Kentucky_Hunting_and_Fishing_Amendment,_House_Bill_1_(2012)
Here!2
<dl><d
Here!2
<p>The
WOO
849
0
0
1243
0
0
https://ballotpedia.org/Maine_Bear_Hunting_Ban_Initiative,_Question_1_(2014)
H

WOO
3359
0
0
5595
0
0
https://ballotpedia.org/Missouri_Dog_Breeding_Regulation,_Proposition_B_(2010)
Here!2
<dl><d
WOO
4412
0
0
3970
0
0
https://ballotpedia.org/Montana_Animal_Trap_Restrictions_Initiative,_I-177_(2016)
WAH
4924
0
0
4743
2
0
https://ballotpedia.org/North_Dakota_Farming_and_Ranching_Amendment,_Measure_3_(2012)
Here!2
<dl><d
WOO
83
0
0
37
0
0
https://ballotpedia.org/North_Dakota_Prevention_of_Animal_Cruelty_Initiative,_Measure_5_(2012)
Here!2
<dl><d
WAH
246
0
0
292
0
0
https://ballotpedia.org/North_Dakota_Captive_Game_Animal_Killing_Prohibited_Initiative,_Initiated_Statutory_Measure_2_(2010)
https://ballotpedia.org/Ohio_Livestock_Care_Standards,_Amendment_2_(2009)
WOO
2800
0
0
3518
0
0
https://ballotpedia.org/Oregon_Gillnet_Fishing_Initiative,_Measure_81_(2012)
Here!2
<dl><d
WAH
567
0
0
352
0
0
https://ballotpedia.org/Oregon_Wildlife_Trafficking_Prevention,_Measure_100_(2016)
WOO
17011
12
25
748
0
0
https://ballotpedia.org/Texas_Proposition_10,_Transfer_of_Care_of_Law_Enf

WOO
0
0
0
0
0
0
https://ballotpedia.org/Colorado_Obsolete_Alcohol_Constitutional_Provisions,_Referendum_N_(2008)
WOO
0
0
0
0
0
0
https://ballotpedia.org/California_Drug_and_Alcohol_Testing_for_State_Legislators_Act_(2010)
https://ballotpedia.org/South_Dakota_Increase_Alcohol_Tax_Initiative_(2008)
https://ballotpedia.org/South_Dakota_Drinking_Age_Initiative_(2008)
https://ballotpedia.org/Second_Missouri_Initiative_to_Lower_Drinking_Age_(2008)
https://ballotpedia.org/Montana_Alcohol_and_Drug_Treatment_Proposal_(2010)
https://ballotpedia.org/California_Tax_on_Alcohol_Initiative_(2010)
https://ballotpedia.org/Colorado_Beer_and_Wine_Sales_Initiative_(2010)
https://ballotpedia.org/Washington_Excise_Tax_on_Beer_Initiative_(2010)
https://ballotpedia.org/Washington_Minimum_Drinking_Age_Initiative_(2010)
https://ballotpedia.org/First_Missouri_Initiative_to_Lower_Drinking_Age_(2008)
https://ballotpedia.org/North_Dakota_Alcohol_Tax_Increase_Initiative_(2012)
https://ballotpedia.org/Washington_Stat

WOO
2395
0
0
2324
0
0
https://ballotpedia.org/Montana_Loan_Interest_Rate_Limit,_I-164_(2010)
Here!2
<dl><d
Here!2
<p>Off
WOO
3992
0
0
1541
0
0
https://ballotpedia.org/North_Dakota_Pharmacy_Ownership_Initiative,_Measure_7_(2014)
WAH
2662
0
0
1532
0
0
https://ballotpedia.org/Ohio_Payday_Lender_Interest_Rate_Cap,_Referendum_5_(2008)
WOO
2198
1
0
3563
0
0
https://ballotpedia.org/Ohio_Initiated_Monopolies_Amendment,_Issue_2_(2015)
WOO
4846
44
2
6941
0
0
https://ballotpedia.org/Oklahoma_Wine_Retail_Sale,_State_Question_743_(2008)
WOO
0
0
0
0
0
0
https://ballotpedia.org/Oklahoma_State_Question_793,_Right_of_Optometrists_and_Opticians_to_Practice_in_Retail_Establishments_Initiative_(2018)
WOO
3143
0
0
3494
0
0
https://ballotpedia.org/Oregon_Mandatory_Labeling_of_GMOs_Initiative,_Measure_92_(2014)
Here!2
<p>Bel
WAH
3642
0
1
6078
0
0
https://ballotpedia.org/South_Dakota_Securities_Practices_and_Transactions,_Initiative_9_(2008)
Here!2
<p><i>
WAH
960
1
0
1745
0
0
https://ballotpedia.org/South_Dak

https://ballotpedia.org/California_Foreclosure_Modification_Act_(2012)
https://ballotpedia.org/Florida_Genetically_Engineered_Food_Label_Amendment_(2012)
https://ballotpedia.org/California_Limit_on_Prices_Set_by_Private_Hospitals_(2012)
https://ballotpedia.org/California_Pollution_Producers_To_Pay_for_Pollution_Mitigation_(2012)
https://ballotpedia.org/California_%22Corporations_Amendment%22_Initiative_(2012)
https://ballotpedia.org/California_Mortgage_Lenders_Required_to_Reduce_Loan_Balances_to_Current_Value_of_Home_(2012)
https://ballotpedia.org/Massachusetts_Charitable_Solicitation_Regulation_Initiative_(4)_(2012)
https://ballotpedia.org/Massachusetts_Auto_Sale_Cancellation_Initiative_(2012)
https://ballotpedia.org/Missouri_Payday_Loan_Initiative_(2012)
https://ballotpedia.org/California_%22Patient_Reasonable_Access_Act%22_(2012)
https://ballotpedia.org/California_%22Sustainable_California_State_Bank%22_(2012)
https://ballotpedia.org/Massachusetts_Whale_and_Sea_Turtle_Protection_Ini

...

https://ballotpedia.org/Washington_Supermajority_Vote_Requirement,_Initiative_485_(2012)
https://ballotpedia.org/West_Virginia_Tangible_Personal_Property_Tax_Exemption_and_Rate_Reduction_Amendment_(2012)
https://ballotpedia.org/North_Carolina_Supermajority_Vote_Required_to_Levy_Taxes_(2012)
https://ballotpedia.org/North_Carolina_Taxpayer_Bill_of_Rights_Amendment_(2012)
https://ballotpedia.org/California_%22Sustainable_California_State_Bank%22_(2012)
https://ballotpedia.org/California_Tax_on_Oil_and_Natural_Gas_(2014)
https://ballotpedia.org/California_Elimination_of_Religion-Based_Property_Tax_Exemptions_(2014)
https://ballotpedia.org/Michigan_Graduated_Income_Tax_Amendment_(2012)
https://ballotpedia.org/Michigan_Local_Sales_Tax_Amendment_(2012)
https://ballotpedia.org/Missouri_Income_Tax_Credit_Initiative_(2012)
https://ballotpedia.org/Missouri_Personal_Property_Tax_Amendment_(2012)
https://ballotpedia.org/Missouri_Income_Tax_Replacement_Initiative_(2012)
https://ballotpedia.org/Oklah

https://ballotpedia.org/Washington_Repeal_Business_and_Occupation_Taxes_Measure,_Initiative_1365_(2015)
https://ballotpedia.org/Washington_Repeal_Motor_Vehicle_Fuel_Taxes_Measure,_Initiative_1367_(2015)
https://ballotpedia.org/Washington_Maximum_Regular_Property_Tax_Levies_Measure,_Initiative_1368_(2015)
https://ballotpedia.org/Washington_Repeal_Real_Estate_Excise_Taxes_Measure,_Initiative_1369_(2015)
https://ballotpedia.org/Washington_Decrease_Sales_Tax_Measure,_Initiative_1370_(2015)
https://ballotpedia.org/Washington_Employee_Paycheck_Deductions_Measure,_Initiative_1371_(2015)
https://ballotpedia.org/Washington_Decrease_Sales_Tax_Measure,_Initiative_1377_(2015)
https://ballotpedia.org/Washington_Taxes_on_Retail_Checkout_Bags_Measure,_Initiative_1378_(2015)
https://ballotpedia.org/Washington_Decrease_Sales_Tax_Measure,_Initiative_1379_(2015)
https://ballotpedia.org/Washington_Corruption_Tax_Measure,_Initiative_1381_(2015)
https://ballotpedia.org/Washington_Use_of_Public_Resources_for

https://ballotpedia.org/California_No_Taxes_on_Residents_Without_Students_for_Public_Schools_Initiative_(2018)
https://ballotpedia.org/Washington_Carbon_Emissions_Mitigation_Fee_Initiative_(2017)
https://ballotpedia.org/California_Homeowners_and_Renters_Tax_Credit_Initiative_(2018)
https://ballotpedia.org/Louisiana_Flat_Income_Tax_and_Eliminate_Federal_Income_Tax_Deduction_for_Corporate_Liability_Amendment_(2017)
https://ballotpedia.org/California_Two-Thirds_Vote_for_State_and_Local_Revenue_Increases_Initiative_(2018)
https://ballotpedia.org/Washington_Repeal_Excise_Taxes_and_Reduce_Minimum_Wage_Initiative_(2018)
https://ballotpedia.org/Oklahoma_Increased_Tax_on_Oil_and_Gas_Wells_for_Education_Funding_Initiative,_State_Question_795_(2018)
https://ballotpedia.org/Arizona_Eliminate_Sales_Tax_Exemptions_for_School_Funding_and_Teacher_Salaries_Initiative_(2018)
https://ballotpedia.org/California_Estate_Tax_for_College_Access_Fund_Initiative_(2018)
https://ballotpedia.org/Nevada_Depreciatio

https://ballotpedia.org/Michigan_Graduated_Income_Tax_Initiative_(2020)
https://ballotpedia.org/Missouri_Capital_Gains_Tax_for_Solar_and_Wind_Energy_Projects_Initiative_(2020)
https://ballotpedia.org/Missouri_Prohibit_Taxes_on_Personal_Property_Initiative_(2020)
https://ballotpedia.org/Missouri_Corporate_Franchise_Tax_Initiative_(2020)
https://ballotpedia.org/Arkansas_Eliminate_Sales_Tax_on_Used_Cars_Sold_for_$20,000_or_Less_Initiative_(2020)
https://ballotpedia.org/Arkansas_Prohibit_State_Sales_Tax_and_Personal_Income_Tax_Initiative_(2020)
https://ballotpedia.org/Georgia_Allow_Local_Property_Tax_Referendums_for_Education_Funding_Amendment_(2020)
https://ballotpedia.org/Louisiana_Capital_Investment_Projects_Property_Tax_Exemption_Amendment_(2020)
https://ballotpedia.org/Louisiana_Tax_Exemptions_for_Capital_Investment_Projects_and_Non-residential_Immovable_Property_Amendment_(2020)
https://ballotpedia.org/California_Property_Tax_Transfers_and_Exemptions_Initiative_(2020)
https://ballotp

WOO
2190
4
2
564
0
0
https://ballotpedia.org/Texas_Proposition_1,_Property_Tax_Exemption_for_Partially_Disabled_Veteran_with_Donated_House_Amendment_(2017)
WOO
1610
1
1
392
0
0
https://ballotpedia.org/Texas_Proposition_8,_Homestead_Tax_Exemption_for_Surviving_Spouses_of_Military_Fatally_Injured_in_the_Line_of_Duty_Amendment_(2021)
WOO
209
1
2
117
0
0
https://ballotpedia.org/Utah_Military_Property_Tax_Exemption_Amendment,_Constitutional_Amendment_B_(2012)
Here!2
<dl><d
Here!2
<p>The
WOO
0
0
0
0
0
0
https://ballotpedia.org/Utah_Constitutional_Amendment_A,_Active_Military_Property_Tax_Exemption_Measure_(2018)
WOO
1115
2
0
288
0
0
https://ballotpedia.org/Virginia_Property_Tax_Exemption_for_Veterans,_Question_2_(2010)
Here!2
<dl><d
WOO
1050
0
0
46
0
0
https://ballotpedia.org/Virginia_Property_Tax_Exemption_for_Surviving_Spouses_of_Armed_Forces_Amendment_(2014)
WOO
1229
6
1
523
0
0
https://ballotpedia.org/Virginia_Question_2,_Remove_Restriction_on_Residence_for_Surviving_Spouse_of_Disabled_V

In [133]:
# Local ballot measure scraper
# Get topic while scraping!

measure_by_topic = 'https://ballotpedia.org/Political_topics_addressed_in_local_ballot_measure_elections'

topics_page = requests.get(measure_by_topic)
topics_soup = BeautifulSoup(topics_page.content, 'html.parser')

topic_table = topics_soup.find('tr')
topic_links = topic_table.find_all("a", href=True)

[<a href="#City"><span class="tocnumber">1</span> <span class="toctext">City</span></a>, <a href="#County"><span class="tocnumber">2</span> <span class="toctext">County</span></a>, <a href="#District"><span class="tocnumber">3</span> <span class="toctext">District</span></a>, <a href="#Energy_and_Environment"><span class="tocnumber">4</span> <span class="toctext">Energy and Environment</span></a>, <a href="#Local_business_and_labor"><span class="tocnumber">5</span> <span class="toctext">Local business and labor</span></a>, <a href="#Local_governance"><span class="tocnumber">6</span> <span class="toctext">Local governance</span></a>, <a href="#Local_social_issues"><span class="tocnumber">7</span> <span class="toctext">Local social issues</span></a>, <a href="#School"><span class="tocnumber">8</span> <span class="toctext">School</span></a>, <a href="#Local_Taxes"><span class="tocnumber">9</span> <span class="toctext">Local Taxes</span></a>, <a href="#Other"><span class="tocnumber">10</sp

In [156]:
# can first get links for each topic -- array of dictionaries with a topic and list of links

categories = []
for group in topic_links:
    print(group)
    if (group['href'][0] == "#"):
        continue;
    
    time.sleep(0.25)
    group_page = requests.get('https://ballotpedia.org' + group['href'] + '#By_state')
    group_soup = BeautifulSoup(group_page.content, 'html.parser')
    
    by_state = group_soup.find("ol")
    if (by_state is None):
        continue
    state_links = by_state("a", href=True)
    
    category = {"topic": group['href'], "measures": []}
    for state_link in state_links:
        print(state_link['href'])
        category["measures"].append(state_link['href'])
    categories.append(category)

<a href="#City"><span class="tocnumber">1</span> <span class="toctext">City</span></a>
<a href="#County"><span class="tocnumber">2</span> <span class="toctext">County</span></a>
<a href="#District"><span class="tocnumber">3</span> <span class="toctext">District</span></a>
<a href="#Energy_and_Environment"><span class="tocnumber">4</span> <span class="toctext">Energy and Environment</span></a>
<a href="#Local_business_and_labor"><span class="tocnumber">5</span> <span class="toctext">Local business and labor</span></a>
<a href="#Local_governance"><span class="tocnumber">6</span> <span class="toctext">Local governance</span></a>
<a href="#Local_social_issues"><span class="tocnumber">7</span> <span class="toctext">Local social issues</span></a>
<a href="#School"><span class="tocnumber">8</span> <span class="toctext">School</span></a>
<a href="#Local_Taxes"><span class="tocnumber">9</span> <span class="toctext">Local Taxes</span></a>
<a href="#Other"><span class="tocnumber">10</span> <span 

/Butte_County,_California,_Orchard_Hospital_Parcel_Tax,_Measure_M1_(November_2016)
/Calaveras_County,_California,_Marijuana_Cultivation_Tax,_Measure_C_(November_2016)
/Contra_Costa_County,_California,_Sales_Tax_for_Transportation_Projects,_Measure_X_(November_2016)
/Healdsburg,_California,_Hotel_Tax_Adjustment,_Measure_S_(November_2016)
/Humboldt_County,_California,_Marijuana_Cultivation_Tax,_Measure_S_(November_2016)
/Humboldt_County,_California,_Transportation_Sales_Tax,_Measure_U_(November_2016)
/Inyo_County,_California,_Marijuana_Business_Tax,_Measure_I_(November_2016)
/Kern_County,_California,_Library_Sales_Tax,_Measure_F_(June_2016)
/Kings_County,_California,_Sales_Tax_for_Public_Safety_Services,_Measure_K_(November_2016)
/Lake_County,_California,_Marijuana_Tax,_Measure_C_(November_2016)
/Los_Angeles_County,_California,_Parks_and_Open_Space_Parcel_Tax,_Measure_A_(November_2016)
/Los_Angeles_County,_California,_Sales_Tax,_Measure_M_(November_2016)
/Marin_County,_California,_Sales_

/Anchorage,_Alaska,_Proposition_5,_Parks_and_Recreation_Service_Area_Bond_Issue_and_Tax_Cap_Increase_(April_2019)
/Anchorage,_Alaska,_Proposition_6,_Fire_Service_Area_Bond_Issue_(April_2019)
/Anchorage,_Alaska,_Proposition_7,_Metropolitan_Police_Service_Area_Bond_Issue_(April_2019)
/Anchorage,_Alaska,_Proposition_4,_Anchorage_Roads_and_Drainage_Service_Area_Bond_Issue_and_Tax_Cap_Increase_(April_2019)
<a href="/District_tax_on_the_ballot" title="District tax on the ballot">District tax on the ballot</a>
/Maricopa_County_Special_Health_Care_District,_Arizona,_Proposition_449,_Continue_Property_Tax_Levy_(November_2020)
<a href="/Special_district_governance_on_the_ballot" title="Special district governance on the ballot">Special district governance on the ballot</a>
<a href="/Local_fracking_on_the_ballot" title="Local fracking on the ballot">Local fracking on the ballot</a>
/Butte_County,_California,_Fracking_Ban_Initiative,_Measure_E_(June_2016)
/Monterey_County,_California,_Ban_on_Oil_a

/Tucson,_Arizona,_Mayor_and_City_Council_Salary_Increase_Charter_Amendment,_Proposition_406_(November_2017)
/Phoenix,_Arizona,_Proposition_415,_Citizens%E2%80%99_Commission_on_Salaries_for_Elected_City_Officials_Terms_and_Meeting_Dates_Charter_Amendment_(August_2018)
/Tucson,_Arizona,_Proposition_409,_Mayor_and_City_Council_Salary_Increase_Charter_Amendment_(November_2019)
/Tucson,_Arizona,_Proposition_410,_Increases_to_Mayoral_and_City_Council_Compensation_Amendment_(November_2021)
<a href="/Local_alcohol_on_the_ballot" title="Local alcohol on the ballot">Local alcohol on the ballot</a>
/Anchorage,_Alaska,_Proposition_9,_Alcoholic_Beverage_Sales_Tax_for_Behavioral_Health,_Homelessness_Services,_and_Prohibited_Campsite_Removal_Charter_Amendment_(April_2019)
/Anchorage,_Alaska,_Proposition_13,_Alcohol_Sales_Tax_Amendment_(April_2020)
<a href="/Local_English_language_on_the_ballot" title="Local English language on the ballot">Local English language on the ballot</a>
<a href="/Local_envir

/Maricopa_County_Special_Health_Care_District,_Arizona,_Proposition_449,_Continue_Property_Tax_Levy_(November_2020)
<a href="/Gann_overrides_on_the_ballot" title="Gann overrides on the ballot">Gann overrides on the ballot</a>
/Anchorage,_Alaska,_Proposition_5,_Parks_and_Recreation_Service_Area_Bond_Issue_and_Tax_Cap_Increase_(April_2019)
/Anchorage,_Alaska,_Proposition_2,_Safety_and_Transportation_Bond_Issue_and_Tax_Cap_Increase_(April_2019)
/Anchorage,_Alaska,_Proposition_4,_Anchorage_Roads_and_Drainage_Service_Area_Bond_Issue_and_Tax_Cap_Increase_(April_2019)
<a href="/Local_business_tax_on_the_ballot" title="Local business tax on the ballot">Local business tax on the ballot</a>
/Scottsdale,_Arizona,_Question_1,_Transportation_Project_Transaction_Privilege_Tax
<a href="/Local_gas_tax_on_the_ballot" title="Local gas tax on the ballot">Local gas tax on the ballot</a>
/Long_Beach,_California,_Measure_US,_General_Oil_Production_Tax_Increase_(November_2020)
<a href="/Local_hotel_tax_on_th

/Henry_County_school_tax_(2009)
/Birmingham,_Alabama,_Proposition_1,_42-Cent_Property_Tax_Continuation_for_Public_Schools_(October_2019)
/Birmingham,_Alabama,_Proposition_2,_28-Cent_Property_Tax_Continuation_for_Public_Schools_(October_2019)
/Birmingham,_Alabama,_Proposition_3,_28-Cent_Property_Tax_Continuation_to_Pay_Public_School_Bonds_(October_2019)
<a href="/Local_sales_tax_on_the_ballot" title="Local sales tax on the ballot">Local sales tax on the ballot</a>
/Anchorage,_Alaska,_Proposition_9,_Alcoholic_Beverage_Sales_Tax_for_Behavioral_Health,_Homelessness_Services,_and_Prohibited_Campsite_Removal_Charter_Amendment_(April_2019)
/Juneau,_Alaska,_Proposition_1,_Sales_Tax_Renewal_(October_2021)
<a href="/Local_school_tax_on_the_ballot" title="Local school tax on the ballot">Local school tax on the ballot</a>
/Henry_County_school_tax_(2009)
<a href="/Local_tobacco_tax_on_the_ballot" title="Local tobacco tax on the ballot">Local tobacco tax on the ballot</a>
/Oakview_Village_Tobacco_Ta

/Salinas,_California,_Fireworks_Legalization_Initiative,_Measure_C_(June_2016)
/Duarte,_California,_Measure_F,_Fireworks_Ban_(November_2018)
/Fullerton,_California,_Measure_U,_Fireworks_Ordinance_(November_2020)
/Rohnert_Park,_California,_Measure_D,_Fireworks_Ban_Referendum_(September_2021)
<a href="/Local_fluoridation_on_the_ballot" title="Local fluoridation on the ballot">Local fluoridation on the ballot</a>
/Healdsburg,_California,_Fluoridation_of_Water_Supply,_Measure_T_(November_2016)
<a href="/Local_GMO_on_the_ballot" title="Local GMO on the ballot">Local GMO on the ballot</a>
/Sonoma_County,_California,_Genetically_Modified_Organism_(GMO)_Ban,_Measure_M_(November_2016)
<a class="mw-redirect" href="/Local_health_care_on_the_ballot" title="Local health care on the ballot">Local health care on the ballot</a>
/Maricopa_County_Special_Health_Care_District,_Arizona,_Proposition_449,_Continue_Property_Tax_Levy_(November_2020)
<a href="/Local_hospital_bonds_on_the_ballot" title="Local h

/Anchorage,_Alaska,_Proposition_8,_Transfer_Parkland_Substations_to_Chugach_Electric_Association_(April_2019)
/Anchorage,_Alaska,_Proposition_11,_Current_Assembly_Discretion_for_Lease-Purchase_Agreements_Charter_Amendment_(April_2019)
<a href="/Local_red_light_cameras_on_the_ballot" title="Local red light cameras on the ballot">Local red light cameras on the ballot</a>
/City_of_Tucson_Photographic_Traffic_Enforcement_Ban_Initiative,_Proposition_201_(November_2015)
/City_of_Arlington_Red_Light_Camera_Ban,_Proposition_1_(May_2015)
<a href="/Local_rent_control_on_the_ballot" title="Local rent control on the ballot">Local rent control on the ballot</a>
/San_Francisco,_California,_Increased_Income_Qualifications_for_Affordable_Housing,_Proposition_U_(November_2016)
/Mountain_View,_California,_Rent_Control_City_Charter_Amendment,_Measure_V_(November_2016)
/Mountain_View,_California,_Rent_Stabilization_Tenant-Landlord_Dispute_Ordinance,_Measure_W_(November_2016)
/Richmond,_California,_Rent_Co

In [169]:
# I DON'T THINK THIS GOT USED

wb = Workbook()  # Prepare to save to spreadsheet. NEED TO FIX
local_ballot_measures = wb.add_sheet('Local Ballot Measures')
local_ballot_measures.write(0, 0, "Year");
local_ballot_measures.write(0, 1, "State");
local_ballot_measures.write(0, 2, "Category");
local_ballot_measures.write(0, 3, "Name");
local_ballot_measures.write(0, 4, "Link");
local_ballot_measures.write(0, 5, "Support contributions");
local_ballot_measures.write(0, 6, "Oppose contributions");
local_ballot_measures.write(0, 7, "Yes votes");
local_ballot_measures.write(0, 8, "No votes");
local_ballot_measures.write(0, 9, "Title?");
local_ballot_measures.write(0, 10, "Ballot title");
local_ballot_measures.write(0, 11, "Ballot question");
local_ballot_measures.write(0, 12, "Short ballot title");
local_ballot_measures.write(0, 13, "Long ballot title");
local_ballot_measures.write(0, 14, "Summary?");
local_ballot_measures.write(0, 15, "Ballot summary");
local_ballot_measures.write(0, 16, "Summary and analysis");
local_ballot_measures.write(0, 17, "Republican presidential vote");
local_ballot_measures.write(0, 18, "Democratic presidential vote");

row = 0
for category in categories:
    print(category["topic"])
    if (category["topic"][0] != "/"):
        continue
    for ballot_measure in category["measures"]:
        row += 1
        
        print("https://ballotpedia.org" + ballot_measure)
        initiative_page = requests.get("https://ballotpedia.org" + ballot_measure)
        initiative_soup = BeautifulSoup(initiative_page.content, 'html.parser')

        local_ballot_measures.write(row, 2, "https://ballotpedia.org" + category["topic"])
        
        # get results
        local_ballot_measures.write(row, 4, ballot_measure)
        results = initiative_soup.find(id="Election_results")
        if (results is None):
            continue
        results_start = results.find_parent().find_next_sibling()
        print(results_start)
        try:
            while (str(results_start)[0:6] != "<table"):
                print("Here!2")
                print(str(results_start)[0:6])
                results_start = results_start.find_next_sibling()

            if (results_start.findChildren("td")[6].text == "No" or results_start.findChildren("td")[3].text == "Yes" or results_start.findChildren("td")[6].text[2:] == "No" or results_start.findChildren("td")[3].text[2:] == "Yes"):
                print("WOO")
                local_ballot_measures.write(row, 7, results_start.findChildren("td")[4].text);
                local_ballot_measures.write(row, 8, results_start.findChildren("td")[7].text);
            else:
                print("WAH")
                local_ballot_measures.write(row, 8, results_start.findChildren("td")[4].text);
                local_ballot_measures.write(row, 7, results_start.findChildren("td")[7].text);
        except:
            print("results are weird :/")

            
        # get ballot title
        has_title = "No"
        ballot_title = initiative_soup.find(id="Ballot_title")  # doesn't always exist! might be "ballot question"
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 10, title_text.text);
            has_title = "Yes"

        ballot_title = initiative_soup.find(id="Ballot_question")
        if (ballot_title is not None):
            title_text = ballot_title.find_parent().find_next_sibling().find_next_sibling()

            local_ballot_measures.write(row, 11, title_text.text);
            has_title = "Yes"

        short_ballot_title = initiative_soup.find(id="Short_ballot_title")
        long_ballot_title = initiative_soup.find(id="Long_ballot_title")
        if (short_ballot_title is not None and long_ballot_title is not None):
            short_title_text = short_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 12, title_text.text);
            long_title_text = long_ballot_title.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 13, title_text.text);

            has_title = "Yes"

        local_ballot_measures.write(row, 9, has_title);

        # get ballot summary
        has_summary = "No"
        ballot_summary = initiative_soup.find(id="Ballot_summary")
        if (ballot_summary is not None):
            summary_text = ballot_summary.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 15, summary_text.text);

            has_summary = "Yes"
        summary_and_analysis = initiative_soup.find(id="Summary_and_analysis")
        if (summary_and_analysis is not None):
            summary_and_analysis_text = summary_and_analysis.find_parent().find_next_sibling().find_next_sibling()
            local_ballot_measures.write(row, 16, summary_and_analysis_text.text);

            has_summary = "Yes"

        local_ballot_measures.write(row, 14, has_summary);
    
wb.save("LOCAL initiatives.xls")

/City_bonds_on_the_ballot
https://ballotpedia.org/Anchorage,_Alaska,_Anchorage_Fire_Service_Area_Fire_Protection_Bonds,_Proposition_5,_(April_2017)
<p>This measure needed to win approval from all city voters as well as the sub-set of voters within the Anchorage Fire Service Area (AFSA).
</p>
Here!2
<p>Thi
Here!2
<div s
Here!2
<dl><d
Here!2
<div i
Here!2
<h2><s
Here!2
<h3><s
Here!2
<p>The
results are weird :/
https://ballotpedia.org/Anchorage,_Alaska,_Anchorage_Metropolitan_Police_Service_Area_Facilities_Bonds,_Proposition_6,_(April_2017)
<p>This measure needed to win approval from all city voters as well as the sub-set of voters within the Anchorage Metropolitan Police Service Area (AMPSA).
</p>
Here!2
<p>Thi
Here!2
<div s
Here!2
<dl><d
Here!2
<div i
Here!2
<h2><s
Here!2
<h3><s
Here!2
<p>The
results are weird :/
https://ballotpedia.org/Anchorage,_Alaska,_Anchorage_Parks_and_Recreation_Service_Area_Capital_Improvement_Bonds,_Proposition_3,_(April_2017)
<p>This measure needed to win appr

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0;background: #f9f9f9;border: 1px #aaa solid;border-collapse: collapse;font-size: 95%; max-width:500px;">
<tbody><tr style="text-align: center;background-color:#E9E9E9;">
<th colspan="5">
<p>					Pima  County  Proposition 463
</p>
</th>
</tr>
<tr align="center" style="background-color:#00008B; color: white;">
<td colspan="2" style="width: 17em;">Result</td>
<td style="width: 5em;">Votes</td>
<td style="width: 7em;">Percentage</td>
</tr>
<tr>
<td colspan="2">Yes</td>
<td style="text-align:right;margin-right:0.5em;">161,437</td>
<td style="text-align:right;margin-right:0.5em;">43.96%</td>
</tr>
<tr>
<td colspan="2">
<p> <a title="Defeated">
<img alt="Defeated" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/0/04/No.png" width="16"/>
</a>
<b>No</b>
</p>
</td>
<td style="text-align:right;margin-right:0.5em;"><b>205,760</b></td>
<td style="text-align:right;margin-right

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0; background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 95%;">
<tbody><tr style="text-align: center; background-color:#E9E9E9;"><th colspan="5"> Measure T</th></tr><tr align="center" style="background-color:#00008B; color: white;"><td colspan="2" style="width: 17em">Result</td><td style="width: 5em">Votes</td><td style="width: 7em">Percentage</td></tr><tr><td colspan="2"><a href="/Defeated" title="Defeated"><img alt="Defeated" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/0/04/No.png" width="16"/></a><span style="display:none;">d</span> <b>No</b></td><td style="text-align: right; margin-right: 0.5em"><b>126,895</b></td><td style="text-align: right; margin-right: 0.5em"><b>52.75%</b></td></tr><tr><td colspan="2">Yes</td><td style="text-align: right; margin-right: 0.5em"> 113,664</td><td style="text-align: right; margin-right: 0.5em">

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0;background: #f9f9f9;border: 1px #aaa solid;border-collapse: collapse;font-size: 95%; max-width:500px;">
<tbody><tr style="text-align: center;background-color:#E9E9E9;">
<th colspan="5">
<p>					Fresno  County  Measure A
</p>
</th>
</tr>
<tr align="center" style="background-color:#00008B; color: white;">
<td colspan="2" style="width: 17em;">Result</td>
<td style="width: 5em;">Votes</td>
<td style="width: 7em;">Percentage</td>
</tr>
<tr>
<td colspan="2">
<p> <a title="Approved">
<img alt="Approved" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/7/77/Yes.png" width="16"/>
</a>
<b>Yes</b>
</p>
</td>
<td style="text-align:right;margin-right:0.5em;"><b>69,137</b></td>
<td style="text-align:right;margin-right:0.5em;">59.49%</td>
</tr>
<tr>
<td colspan="2">No</td>
<td style="text-align:right;margin-right:0.5em;">47,070</td>
<td style="text-align:right;margin-right:0.5e

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0; background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 95%;">
<tbody><tr style="text-align: center; background-color:#E9E9E9;"><th colspan="5"> Measure M1</th></tr><tr align="center" style="background-color:#00008B; color: white;"><td colspan="2" style="width: 17em">Result</td><td style="width: 5em">Votes</td><td style="width: 7em">Percentage</td></tr><tr><td colspan="2"><a href="/Defeated" title="Defeated"><img alt="Defeated" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/0/04/No.png" width="16"/></a><span style="display:none;">d</span> <b>No</b></td><td style="text-align: right; margin-right: 0.5em"><b>1,156</b></td><td style="text-align: right; margin-right: 0.5em"><b>49.55%</b></td></tr><tr><td colspan="2">Yes</td><td style="text-align: right; margin-right: 0.5em"> 1,177</td><td style="text-align: right; margin-right: 0.5em"> 50

...

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0;background: #f9f9f9;border: 1px #aaa solid;border-collapse: collapse;font-size: 95%; max-width:500px;">
<tbody><tr style="text-align: center;background-color:#E9E9E9;">
<th colspan="5">
<p>					Anchorage  Proposition No. 4
</p>
</th>
</tr>
<tr align="center" style="background-color:#00008B; color: white;">
<td colspan="2" style="width: 17em;">Result</td>
<td style="width: 5em;">Votes</td>
<td style="width: 7em;">Percentage</td>
</tr>
<tr>
<td colspan="2">
<p> <a title="Approved">
<img alt="Approved" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/7/77/Yes.png" width="16"/>
</a>
<b>Yes</b>
</p>
</td>
<td style="text-align:right;margin-right:0.5em;"><b>39,507</b></td>
<td style="text-align:right;margin-right:0.5em;">53.81%</td>
</tr>
<tr>
<td colspan="2">No</td>
<td style="text-align:right;margin-right:0.5em;">33,916</td>
<td style="text-align:right;margin-right:0

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0; background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 95%;">
<tbody><tr style="text-align: center; background-color:#E9E9E9;"><th colspan="5"> Measure V</th></tr><tr align="center" style="background-color:#00008B; color: white;"><td colspan="2" style="width: 17em">Result</td><td style="width: 5em">Votes</td><td style="width: 7em">Percentage</td></tr><tr><td colspan="2"><a href="/Approved" title="Approved"><img alt="Approved" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/7/77/Yes.png" width="16"/></a><span style="display:none;">a</span> <b>Yes</b></td><td style="text-align: right; margin-right: 0.5em"> <b>15,393</b></td><td style="text-align: right; margin-right: 0.5em"> <b>53.57%</b></td></tr><tr><td colspan="2">  No</td><td align="right">13,341</td><td align="right">46.43%</td></tr></tbody></table>
WOO
https://ballotpedia.org/Mou

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0; background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 95%;">
<tbody><tr style="text-align: center; background-color:#E9E9E9;"><th colspan="5"> Measure JJ</th></tr><tr align="center" style="background-color:#00008B; color: white;"><td colspan="2" style="width: 17em">Result</td><td style="width: 5em">Votes</td><td style="width: 7em">Percentage</td></tr><tr><td colspan="2"><a href="/Approved" title="Approved"><img alt="Approved" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/7/77/Yes.png" width="16"/></a><span style="display:none;">a</span> <b>Yes</b></td><td style="text-align: right; margin-right: 0.5em"> <b>124,282</b></td><td style="text-align: right; margin-right: 0.5em"> <b>75.24%</b></td></tr><tr><td colspan="2">  No</td><td align="right">40,895</td><td align="right">24.76%</td></tr></tbody></table>
WOO
https://ballotpedia.org/H

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0;background: #f9f9f9;border: 1px #aaa solid;border-collapse: collapse;font-size: 95%; max-width:500px;">
<tbody><tr style="text-align: center;background-color:#E9E9E9;">
<th colspan="5">
<p>					St. Helena  Measure F
</p>
</th>
</tr>
<tr align="center" style="background-color:#00008B; color: white;">
<td colspan="2" style="width: 17em;">Result</td>
<td style="width: 5em;">Votes</td>
<td style="width: 7em;">Percentage</td>
</tr>
<tr>
<td colspan="2">Yes</td>
<td style="text-align:right;margin-right:0.5em;">594</td>
<td style="text-align:right;margin-right:0.5em;">40.24%</td>
</tr>
<tr>
<td colspan="2">
<p> <a title="Defeated">
<img alt="Defeated" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/0/04/No.png" width="16"/>
</a>
<b>No</b>
</p>
</td>
<td style="text-align:right;margin-right:0.5em;"><b>882</b></td>
<td style="text-align:right;margin-right:0.5em;"><b>59.7

<table border="1" cellpadding="4" cellspacing="0" style="margin: 1em 1em 1em 0; background: #f9f9f9; border: 1px #aaa solid; border-collapse: collapse; font-size: 95%;">
<tbody><tr style="text-align: center; background-color:#E9E9E9;"><th colspan="5"> Proposition V</th></tr><tr align="center" style="background-color:#00008B; color: white;"><td colspan="2" style="width: 17em">Result</td><td style="width: 5em">Votes</td><td style="width: 7em">Percentage</td></tr><tr><td colspan="2"><a href="/Approved" title="Approved"><img alt="Approved" data-file-height="16" data-file-width="16" height="16" src="https://cdn.ballotpedia.org/images/7/77/Yes.png" width="16"/></a><span style="display:none;">a</span> <b>Yes</b></td><td style="text-align: right; margin-right: 0.5em"> <b>179,639</b></td><td style="text-align: right; margin-right: 0.5em"> <b>54.63%</b></td></tr><tr><td colspan="2">  No</td><td align="right">149,169</td><td align="right">45.37%</td></tr></tbody></table>
WOO
https://ballotpedia.o